<a href="https://colab.research.google.com/github/Fadhlin11/assignment9/blob/main/Building_Recommender_Fadhlin_Sakina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
%%capture
!sudo apt-get update --fix-missing

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
#!wget -q https://downloads.apache.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

!mv spark-3.0.0-bin-hadoop3.2.tgz sparkkk
!tar xf sparkkk
!pip install -q findspark
     

#pip install spark
     

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName('fpgrowth') \
    .getOrCreate()

spark 

In [30]:
complete_dataset_url = 'https://files.grouplens.org/datasets/movielens/ml-latest.zip'   
small_dataset_url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'

In [31]:
import os

datasets_path = os.path.join('/content', 'datasets')

complete_dataset_path = os.path.join(datasets_path, 'ml-latest.zip')
small_dataset_path = os.path.join(datasets_path, 'ml-latest-small.zip')

In [32]:
import urllib
import urllib.request

small_f = urllib.request.urlretrieve(small_dataset_url, small_dataset_path)
complete_f = urllib.request.urlretrieve(complete_dataset_url, complete_dataset_path)   

In [33]:
import zipfile

with zipfile.ZipFile(small_dataset_path, "r") as z:
    z.extractall(datasets_path)

with zipfile.ZipFile(complete_dataset_path, "r") as z:
    z.extractall(datasets_path)

In [34]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate();

small_ratings_file = os.path.join(datasets_path, 'ml-latest-small', 'ratings.csv')

small_ratings_raw_data = sc.textFile(small_ratings_file)
small_ratings_raw_data_header = small_ratings_raw_data.take(1)[0]

In [35]:
small_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()
     

In [36]:
small_ratings_data.take(3)

[('1', '1', '4.0'), ('1', '3', '4.0'), ('1', '6', '4.0')]

In [37]:
small_movies_file = os.path.join(datasets_path, 'ml-latest-small', 'movies.csv')

small_movies_raw_data = sc.textFile(small_movies_file)
small_movies_raw_data_header = small_movies_raw_data.take(1)[0]

small_movies_data = small_movies_raw_data.filter(lambda line: line!=small_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1])).cache()
    
small_movies_data.take(3)

[('1', 'Toy Story (1995)'),
 ('2', 'Jumanji (1995)'),
 ('3', 'Grumpier Old Men (1995)')]

In [38]:
training_RDD, validation_RDD, test_RDD = small_ratings_data.randomSplit([6., 2., 2.])
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [39]:
from pyspark.mllib.recommendation import ALS
import math

seed = 5

# we will use the 3 value as the same as the value suggest in assignment 9 which is 5 10 20

iterations_d = [5, 10 ,15] # iteration value for addtional inner loop 


regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0, 0, 0, 0, 0 ,0 ,0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
best_iteration_value = -1
for rank in ranks:
    for it in iterations_d:
      model = ALS.train(training_RDD, rank, seed=seed, iterations=it,
                        lambda_=regularization_parameter)
      predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
      rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
      error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
      errors[err] = error
      err += 1
      print('For rank ' + str(rank) + ' with iteration value ' + str(it) +' the RMSE is ' + str(error) )
      if error < min_error:
          min_error = error
          best_rank = rank
          best_iteration_value = it

print('The best model was trained with rank ' + str(best_rank) + ' with iteration value ' + str(best_iteration_value))

For rank 4 with iteration value 5 the RMSE is 0.9113636630541257
For rank 4 with iteration value 10 the RMSE is 0.9079129373037642
For rank 4 with iteration value 15 the RMSE is 0.9076518695958723
For rank 8 with iteration value 5 the RMSE is 0.9197243590310956
For rank 8 with iteration value 10 the RMSE is 0.9175857970137057
For rank 8 with iteration value 15 the RMSE is 0.916051478406906
For rank 12 with iteration value 5 the RMSE is 0.9231730315937859
For rank 12 with iteration value 10 the RMSE is 0.9175923959537667
For rank 12 with iteration value 15 the RMSE is 0.9158613827270959
The best model was trained with rank 4 with iteration value 15


In [40]:
predictions.take(3)

[((156, 1084), 3.849157723735146),
 ((294, 1084), 3.168244409500634),
 ((410, 1084), 4.396141476219778)]

In [41]:
rates_and_preds.take(3)

[((1, 101), (5.0, 3.8966263330538444)),
 ((1, 441), (4.0, 4.0072522308389775)),
 ((1, 593), (4.0, 4.729580413113688))]

In [42]:
model = ALS.train(training_RDD, best_rank, seed=seed, iterations=best_iteration_value,
                      lambda_=regularization_parameter)
predictions = model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print('For testing data the RMSE is ' + str(error))

For testing data the RMSE is 0.9087302752623122


In [43]:
# Load the complete dataset file
complete_ratings_file = os.path.join(datasets_path, 'ml-latest', 'ratings.csv')
complete_ratings_raw_data = sc.textFile(complete_ratings_file)
complete_ratings_raw_data_header = complete_ratings_raw_data.take(1)[0]

# Parse
complete_ratings_data = complete_ratings_raw_data.filter(lambda line: line!=complete_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2]))).cache()
    
print("There are " + str(complete_ratings_data.count()) + " recommendations in the complete dataset" )

There are 27753444 recommendations in the complete dataset


In [44]:
training_RDD, test_RDD = complete_ratings_data.randomSplit([7, 3], seed=0)

complete_model = ALS.train(training_RDD, best_rank, seed=seed, 
                           iterations=best_iteration_value, lambda_=regularization_parameter)

In [45]:
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

predictions = complete_model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print('For testing data the RMSE is ' + str(error))

For testing data the RMSE is 0.8325185321688605


In [46]:
complete_movies_file = os.path.join(datasets_path, 'ml-latest', 'movies.csv')
complete_movies_raw_data = sc.textFile(complete_movies_file)
complete_movies_raw_data_header = complete_movies_raw_data.take(1)[0]

# Parse
complete_movies_data = complete_movies_raw_data.filter(lambda line: line!=complete_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1],tokens[2])).cache()

complete_movies_titles = complete_movies_data.map(lambda x: (int(x[0]),x[1]))
    
print("There are " + str(complete_movies_titles.count()) + " movies in the complete dataset" )

There are 58098 movies in the complete dataset


In [47]:
def get_counts_and_averages(ID_and_ratings_tuple):
    nratings = len(ID_and_ratings_tuple[1])
    return ID_and_ratings_tuple[0], (nratings, float(sum(x for x in ID_and_ratings_tuple[1]))/nratings)

movie_ID_with_ratings_RDD = (complete_ratings_data.map(lambda x: (x[1], x[2])).groupByKey())
movie_ID_with_avg_ratings_RDD = movie_ID_with_ratings_RDD.map(get_counts_and_averages)
movie_rating_counts_RDD = movie_ID_with_avg_ratings_RDD.map(lambda x: (x[0], x[1][0]))

In [48]:
new_user_ID = 0

# The format of each line is (userID, movieID, rating)
new_user_ratings = [
     (0,260,9), # Star Wars (1977)
     (0,1,8), # Toy Story (1995)
     (0,16,7), # Casino (1995)
     (0,25,8), # Leaving Las Vegas (1995)
     (0,32,9), # Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
     (0,335,4), # Flintstones, The (1994)
     (0,379,3), # Timecop (1994)
     (0,296,7), # Pulp Fiction (1994)
     (0,858,10) , # Godfather, The (1972)
     (0,50,8) # Usual Suspects, The (1995)
    ]
new_user_ratings_RDD = sc.parallelize(new_user_ratings)
print('New user ratings: ' + str(new_user_ratings_RDD.take(10)))

New user ratings: [(0, 260, 9), (0, 1, 8), (0, 16, 7), (0, 25, 8), (0, 32, 9), (0, 335, 4), (0, 379, 3), (0, 296, 7), (0, 858, 10), (0, 50, 8)]


In [49]:
complete_data_with_new_ratings_RDD = complete_ratings_data.union(new_user_ratings_RDD)

In [50]:
from time import time

t0 = time()
new_ratings_model = ALS.train(complete_data_with_new_ratings_RDD, best_rank, seed=seed, 
                              iterations=best_iteration_value, lambda_=regularization_parameter)
tt = time() - t0

print("New model trained in " + str(round(tt,3)) + " seconds")

New model trained in 267.466 seconds


In [51]:
new_user_ratings_ids = map(lambda x: x[1], new_user_ratings) # get just movie IDs
# keep just those not on the ID list (thanks Lei Li for spotting the error!)
new_user_unrated_movies_RDD = (complete_movies_data.filter(lambda x: x[0] not in new_user_ratings_ids).map(lambda x: (new_user_ID, x[0])))

# Use the input RDD, new_user_unrated_movies_RDD, with new_ratings_model.predictAll() to predict new ratings for the movies
new_user_recommendations_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)

In [52]:
# Transform new_user_recommendations_RDD into pairs of the form (Movie ID, Predicted Rating)
new_user_recommendations_rating_RDD = new_user_recommendations_RDD.map(lambda x: (x.product, x.rating))
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_RDD.join(complete_movies_titles).join(movie_rating_counts_RDD)
new_user_recommendations_rating_title_and_count_RDD.take(3)

[(6216,
  ((7.048225289394679, 'Nowhere in Africa (Nirgendwo in Afrika) (2001)'),
   717)),
 (124320, ((7.464667135977711, 'Once a Thief (1965)'), 1)),
 (83916, ((6.295688644211086, 'Blues in the Night (1941)'), 9))]

In [53]:
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_title_and_count_RDD.map(lambda r: (r[1][0][1], r[1][0][0], r[1][1]))

In [54]:
top_movies = new_user_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2]>=25).takeOrdered(25, key=lambda x: -x[1])

print ('TOP recommended movies (with more than 25 reviews):\n%s' %
        '\n'.join(map(str, top_movies)))

TOP recommended movies (with more than 25 reviews):
('Music for One Apartment and Six Drummers (2001)', 8.910493225785338, 31)
('Rabbit of Seville (1950)', 8.871530175110536, 30)
('"Human Condition III', 8.801406382799769, 91)
('Baseball (1994)', 8.780866761199064, 42)
('Harakiri (Seppuku) (1962)', 8.726752026752713, 679)
("Jim Henson's The Storyteller (1989)", 8.689345632399984, 36)
('Wow! A Talking Fish! (1983)', 8.680845086044542, 47)
('"I', 8.672433189547817, 85)
('Elway To Marino (2013)', 8.661020931696399, 25)
('"Last Lions', 8.632423614869886, 38)
('Rabbit Fire (1951)', 8.62922490345224, 46)
('Cosmos', 8.62800144816191, 157)
('"Godfather', 8.607680523140289, 60904)
('Duck Amuck (1953)', 8.595768736337256, 226)
('"Great War', 8.592974864783855, 31)
('"Lonely Wife', 8.582254977361682, 43)
('"Century of the Self', 8.545297039656248, 213)
('Death on the Staircase (Soupçons) (2004)', 8.545110012745454, 130)
('Crooks in Clover (a.k.a. Monsieur Gangster) (Les tontons flingueurs) (1963)

In [55]:
my_movie = sc.parallelize([(0, 500)]) # Quiz Show (1994)
individual_movie_rating_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)
individual_movie_rating_RDD.take(1)

[Rating(user=0, product=116688, rating=2.033429946186693)]

In [56]:
from pyspark.mllib.recommendation import MatrixFactorizationModel

model_path = os.path.join('..', 'models', 'movie_lens_als')

# Save and load model
model.save(sc, model_path)
same_model = MatrixFactorizationModel.load(sc, model_path)